In [12]:
import pandas as pd
import numpy as np
import sys
import os

# Allows to get the module in utils
sys.path.append(os.path.abspath(".."))


from utils.load import load_data
from utils.preprocessing import preprocess
from utils.split import split_data
from utils.seed_definition import set_seed

from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
# from sklearn.compose import ColumnTransformer

import tensorflow as tf

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
import optuna

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.metrics import roc_auc_score, recall_score
from sklearn.utils import class_weight
import numpy as np


In [2]:
set_seed(42)

In [3]:
# Load data
df = load_data()

# Split and preprocess datas set
X_train, X_val, X_test, y_train, y_val, y_test = split_data(df)
X_train_processed, X_test_processed, X_val_processed, y_test_encoding, y_train_encoded, y_val_encoding, pipeline, le = preprocess(X_train, X_val, X_test, y_train, y_val, y_test)

In [4]:
X_train_dense = X_train_processed.toarray() if hasattr(X_train_processed, "toarray") else X_train_processed
X_val_dense = X_val_processed.toarray() if hasattr(X_val_processed, "toarray") else X_val_processed
X_test_dense = X_test_processed.toarray() if hasattr(X_test_processed, "toarray") else X_test_processed

In [5]:
y_train_vector = y_train_encoded.reshape(-1)
y_val_vector = y_val_encoding.reshape(-1)
y_test_vector = y_test_encoding.reshape(-1)

In [6]:
input_dim = X_train_dense.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy', 'recall', 'auc']
)

model.summary()

2025-05-26 09:28:25.327345: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-05-26 09:28:25.327461: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-05-26 09:28:25.327488: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: utilisateur-ThinkPad-P53
2025-05-26 09:28:25.327503: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] hostname: utilisateur-ThinkPad-P53
2025-05-26 09:28:25.327769: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:197] libcuda reported version is: 550.144.3
2025-05-26 09:28:25.327839: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:201] kernel re

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,209 (24.25 KB)

 Trainable params: 6,209 (24.25 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
checkpoint_path = "checkpoints/churn_models.keras"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    mode='max',  
    save_best_only=True,# sauvegarde le modèle qui maximise l’accuracy de validation
    verbose=1
)

In [8]:
history = model.fit(
    X_train_dense, y_train_vector,
    validation_data=(X_val_dense, y_val_vector),
    epochs=20,
    batch_size=16,
    callbacks=[model_ckpt],
    verbose=1
)

Epoch 1/20
246/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7725 - auc: 0.7846 - loss: 0.4649 - recall: 0.3828
Epoch 1: val_accuracy improved from -inf to 0.78971, saving model to checkpoints/churn_models.keras
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7743 - auc: 0.7888 - loss: 0.4628 - recall: 0.3956 - val_accuracy: 0.7897 - val_auc: 0.8355 - val_loss: 0.4490 - val_recall: 0.3579
Epoch 2/20
253/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7988 - auc: 0.8376 - loss: 0.4196 - recall: 0.4904  
Epoch 2: val_accuracy improved from 0.78971 to 0.79148, saving model to checkpoints/churn_models.keras
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7981 - auc: 0.8374 - loss: 0.4206 - recall: 0.4919 - val_accuracy: 0.7915 - val_auc: 0.8371 - val_loss: 0.4438 - val_recall: 0.3512
Epoch 3/20
235/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8030 - auc: 0.8409 - loss: 0.4142 - recall: 0.4517
Epoch 3: val_accuracy improved from 0.79148 to 0.79503, saving model t

In [9]:

def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    epochs = trial.suggest_int('epochs', 10, 50)
    n_layers = trial.suggest_int('n_layers', 1, 3)
    n_units = trial.suggest_int('n_units', 32, 128, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])

    # Optimizer mapping
    optimizer = {
        'adam': Adam(learning_rate=learning_rate),
        'rmsprop': RMSprop(learning_rate=learning_rate)
    }[optimizer_name]

    # Compute class weights
    class_weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train_vector),
        y=y_train_vector
    )
    class_weights = dict(enumerate(class_weights))

    # Build model
    model = tf.keras.Sequential()
    model.add(Input(shape=(X_train_dense.shape[1],)))
    for _ in range(n_layers):
        model.add(Dense(n_units, activation=activation))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['recall']
    )

    # Early stopping
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_auc', mode='max', patience=5,
        restore_best_weights=True, verbose=0
    )

    # Train model
    model.fit(
        X_train_dense, y_train_vector,
        validation_data=(X_val_dense, y_val_vector),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stop],
        class_weight=class_weights
    )

    # Evaluate
    y_pred_proba = model.predict(X_val_dense)
    auc = roc_auc_score(y_val_vector, y_pred_proba)
    return auc


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42), storage="sqlite:///db.sqlite3", study_name='test_seed')
study.optimize(objective, n_trials=30)

[I 2025-05-26 09:29:34,995] A new study created in RDB with name: test_seed
/tmp/ipykernel_178822/2772100943.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: loss,recall,val_loss,val_recall
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-05-26 09:29:41,921] Trial 0 finished with value: 0.8449339989982713 and parameters: {'learning_rate': 0.0001329291894316216, 'batch_size': 16, 'epochs': 16, 'n_layers': 1, 'n_units': 32, 'dropout_rate': 0.4330880728874676, 'activation': 'tanh', 'optimizer': 'rmsprop'}. Best is trial 0 with value: 0.8449339989982713.
/tmp/ipykernel_178822/2772100943.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: loss,recall,val_loss,val_recall
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-05-26 09:29:51,220] Trial 1 finished with value: 0.8262323687654501 and parameters: {'learning_rate': 0.00314288089084011, 'batch_size': 16, 'epochs': 22, 'n_layers': 2, 'n_units': 64, 'dropout_rate': 0.14561457009902096, 'activation': 'relu', 'optimizer': 'rmsprop'}. Best is trial 0 with value: 0.8449339989982713.
/tmp/ipykernel_178822/2772100943.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: loss,recall,val_loss,val_recall
  current = self.get_monitor_value(logs)


...

[I 2025-05-26 09:33:20,823] Trial 27 finished with value: 0.8485248735721325 and parameters: {'learning_rate': 0.000606514058090697, 'batch_size': 64, 'epochs': 18, 'n_layers': 1, 'n_units': 96, 'dropout_rate': 0.10775445381745677, 'activation': 'tanh', 'optimizer': 'adam'}. Best is trial 2 with value: 0.8499184075743622.
/tmp/ipykernel_178822/2772100943.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: loss,recall,val_loss,val_recall
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-05-26 09:33:26,205] Trial 28 finished with value: 0.8489166787843536 and parameters: {'learning_rate': 0.0012882687203160169, 'batch_size': 64, 'epochs': 27, 'n_layers': 2, 'n_units': 128, 'dropout_rate': 0.34002422398680154, 'activation': 'tanh', 'optimizer': 'adam'}. Best is trial 2 with value: 0.8499184075743622.
/tmp/ipykernel_178822/2772100943.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: loss,recall,val_loss,val_recall
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-05-26 09:33:39,013] Trial 29 finished with value: 0.8458024332315447 and parameters: {'learning_rate': 0.00029461426092931647, 'batch_size': 16, 'epochs': 33, 'n_layers': 1, 'n_units': 96, 'dropout_rate': 0.2372276176062828, 'activation': 'tanh', 'optimizer': 'rmsprop'}. Best is trial 2 with value: 0.8499184075743622.


In [11]:
print("Best trial:")
trial = study.best_trial
print(f"AUC: {trial.value}")
print("Params:")
for key, value in trial.params.items():
    print(f"  {key}: {value}")


Best trial:
AUC: 0.8499184075743622
Params:
  learning_rate: 0.00023345864076016249
  batch_size: 16
  epochs: 34
  n_layers: 1
  n_units: 96
  dropout_rate: 0.08526206184364576
  activation: tanh
  optimizer: adam


In [ ]:

def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    epochs = trial.suggest_int('epochs', 10, 50)
    n_layers = trial.suggest_int('n_layers', 1, 3)
    n_units = trial.suggest_int('n_units', 32, 128, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])

    # Optimizer mapping
    optimizer = {
        'adam': Adam(learning_rate=learning_rate),
        'rmsprop': RMSprop(learning_rate=learning_rate)
    }[optimizer_name]

    # Compute class weights
    class_weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train_vector),
        y=y_train_vector
    )
    class_weights = dict(enumerate(class_weights))

    # Build model
    model = tf.keras.Sequential()
    model.add(Input(shape=(X_train_dense.shape[1],)))
    for _ in range(n_layers):
        model.add(Dense(n_units, activation=activation))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['recall']
    )

    # Early stopping
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='recall', mode='max', patience=5,
        restore_best_weights=True, verbose=0
    )

    # Train model
    model.fit(
        X_train_dense, y_train_vector,
        validation_data=(X_val_dense, y_val_vector),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stop],
        class_weight=class_weights
    )

    # Evaluate
    y_pred_proba = model.predict(X_val_dense)
    y_pred = (y_pred_proba > 0.5).astype(int)
    recall = recall_score(y_val_vector, y_pred)
    return recall


In [16]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42), storage="sqlite:///db.sqlite3", study_name='maximize_recall1')
study.optimize(objective, n_trials=30)

[I 2025-05-26 09:45:06,948] A new study created in RDB with name: maximize_recall1


/tmp/ipykernel_178822/2120121295.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: loss,recall,val_loss,val_recall
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-05-26 09:45:13,317] Trial 0 finished with value: 0.7859531772575251 and parameters: {'learning_rate': 0.0001329291894316216, 'batch_size': 16, 'epochs': 16, 'n_layers': 1, 'n_units': 32, 'dropout_rate': 0.4330880728874676, 'activation': 'tanh', 'optimizer': 'rmsprop'}. Best is trial 0 with value: 0.7859531772575251.
/tmp/ipykernel_178822/2120121295.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: loss,recall,val_loss,val_recall
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


...

[I 2025-05-26 09:49:52,416] Trial 27 finished with value: 0.7926421404682275 and parameters: {'learning_rate': 2.249166001803003e-05, 'batch_size': 32, 'epochs': 42, 'n_layers': 1, 'n_units': 64, 'dropout_rate': 0.4949267329198006, 'activation': 'relu', 'optimizer': 'rmsprop'}. Best is trial 2 with value: 0.8093645484949833.
/tmp/ipykernel_178822/2120121295.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: loss,recall,val_loss,val_recall
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


[I 2025-05-26 09:49:59,066] Trial 28 finished with value: 0.7558528428093646 and parameters: {'learning_rate': 1.905259069825371e-05, 'batch_size': 64, 'epochs': 36, 'n_layers': 3, 'n_units': 96, 'dropout_rate': 0.49548883189341014, 'activation': 'tanh', 'optimizer': 'rmsprop'}. Best is trial 2 with value: 0.8093645484949833.
/tmp/ipykernel_178822/2120121295.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: loss,recall,val_loss,val_recall
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-05-26 09:50:17,212] Trial 29 finished with value: 0.7859531772575251 and parameters: {'learning_rate': 3.047732817556332e-05, 'batch_size': 16, 'epochs': 49, 'n_layers': 1, 'n_units': 64, 'dropout_rate': 0.0045332811460873645, 'activation': 'tanh', 'optimizer': 'rmsprop'}. Best is trial 2 with value: 0.8093645484949833.
